In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [154]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [155]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [156]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [157]:
display(train.shape)
display(test.shape)

(891, 12)

(418, 11)

In [158]:
display(train.dtypes)

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [159]:
display(train['Pclass'].unique())
display(train.groupby(['Pclass'])[['Survived']].apply(lambda x: x.count()))

array([3, 1, 2])

,Survived
Pclass,
1,216
2,184
3,491


In [160]:
display(train.groupby(['Sex'])[['Survived']].apply(lambda x: x.count()))

,Survived
Sex,
female,314
male,577


In [161]:
display(train['Age'].describe())
# Define age bins
bins = [0, 18, 30, 40, 50, 60, 70, 80]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80']

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [162]:
train['Age Group'] = pd.cut(train['Age'], bins=bins, labels=labels, right=False)
test['Age Group'] = pd.cut(test['Age'], bins=bins, labels=labels, right=False)
display(train.groupby(['Age Group'])[['Survived']].apply(lambda x: x.count()))
display(test.head())

/tmp/ipykernel_30/864930245.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  display(train.groupby(['Age Group'])[['Survived']].apply(lambda x: x.count()))


,Survived
Age Group,
0-18,113
19-30,271
31-40,167
41-50,89
51-60,48
61-70,19
71-80,6


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,31-40
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,41-50
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,61-70
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,19-30
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,19-30


In [163]:

display(train['Embarked'].unique())
embarked_mapping = {
    'nan': 0,
     np.nan: 0,
    'S': 1,
    'C': 2,
    'Q': 3
}
train_plne = train.copy()
test_plne = test.copy()
train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
train_median_age = train_plne['Age'].median()
test_median_age = test_plne['Age'].median()

# Impute NaN values in the Age column with the median age
train_plne['Age'].fillna(train_median_age, inplace=True)
test_plne['Age'].fillna(test_median_age, inplace=True)
display(train_plne.head())
display(test_plne.head())

array(['S', 'C', 'Q', nan], dtype=object)

/tmp/ipykernel_30/854410865.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_plne['Embarked'] = train_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/854410865.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_plne['Embarked'] = test_plne['Embarked'].replace(embarked_mapping)
/tmp/ipykernel_30/854410865.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method wil

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,1,19-30
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,2,31-40
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,19-30
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,1,31-40
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,1,31-40


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age Group
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,3,31-40
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,1,41-50
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,3,61-70
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,1,19-30
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,1,19-30


In [164]:
test['Embarked'].unique()

array(['Q', 'S', 'C'], dtype=object)

In [165]:
train_plne = pd.get_dummies(train_plne, columns=['Sex','Embarked','Pclass','Age Group'], prefix=['Sex','Embarked','Pclass','Age Group'], dtype=int)
test_plne = pd.get_dummies(test_plne, columns=['Sex','Embarked','Pclass', 'Age Group'], prefix=['Sex','Embarked','Pclass','Age Group'], dtype=int)

In [166]:
train_plne.head()

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,...,Pclass_1,Pclass_2,Pclass_3,Age Group_0-18,Age Group_19-30,Age Group_31-40,Age Group_41-50,Age Group_51-60,Age Group_61-70,Age Group_71-80
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,...,0,0,1,0,1,0,0,0,0,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,...,1,0,0,0,0,1,0,0,0,0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,...,0,0,1,0,1,0,0,0,0,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,1,...,1,0,0,0,0,1,0,0,0,0
4,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,...,0,0,1,0,0,1,0,0,0,0


In [167]:
nan_counts = train_plne.isna().sum()
display(nan_counts)
nan_counts = test_plne.isna().sum()
display(nan_counts)

PassengerId          0
Survived             0
Name                 0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 0
Cabin              687
Sex_female           0
Sex_male             0
Embarked_0           0
Embarked_1           0
Embarked_2           0
Embarked_3           0
Pclass_1             0
Pclass_2             0
Pclass_3             0
Age Group_0-18       0
Age Group_19-30      0
Age Group_31-40      0
Age Group_41-50      0
Age Group_51-60      0
Age Group_61-70      0
Age Group_71-80      0
dtype: int64

PassengerId          0
Name                 0
Age                  0
SibSp                0
Parch                0
Ticket               0
Fare                 1
Cabin              327
Sex_female           0
Sex_male             0
Embarked_1           0
Embarked_2           0
Embarked_3           0
Pclass_1             0
Pclass_2             0
Pclass_3             0
Age Group_0-18       0
Age Group_19-30      0
Age Group_31-40      0
Age Group_41-50      0
Age Group_51-60      0
Age Group_61-70      0
Age Group_71-80      0
dtype: int64

In [168]:
train_plne.dtypes

PassengerId          int64
Survived             int64
Name                object
Age                float64
SibSp                int64
Parch                int64
Ticket              object
Fare               float64
Cabin               object
Sex_female           int64
Sex_male             int64
Embarked_0           int64
Embarked_1           int64
Embarked_2           int64
Embarked_3           int64
Pclass_1             int64
Pclass_2             int64
Pclass_3             int64
Age Group_0-18       int64
Age Group_19-30      int64
Age Group_31-40      int64
Age Group_41-50      int64
Age Group_51-60      int64
Age Group_61-70      int64
Age Group_71-80      int64
dtype: object

Check if the age group is a feature that can be used, dummy variable
Cabin seems to have 40% values missing but maybe useful to show likelihood of survival, can be a dummy variable
Maybe PClass should be a dummy variable
We should also look at the fares and see if passengers with higher or lower fare had more probability of survival maybe can be dummy variable based on range.

In [169]:
train_plne.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Sex_female', 'Sex_male', 'Embarked_0', 'Embarked_1',
       'Embarked_2', 'Embarked_3', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Age Group_0-18', 'Age Group_19-30', 'Age Group_31-40',
       'Age Group_41-50', 'Age Group_51-60', 'Age Group_61-70',
       'Age Group_71-80'],
      dtype='object')

In [176]:
drop_columns_train = ['Name','Age','SibSp','Parch','Fare','Ticket','Cabin','Embarked_0']
drop_columns_test = ['Name','Age','SibSp','Parch','Fare','Ticket','Cabin']
train_plne.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Sex_female', 'Sex_male', 'Embarked_0', 'Embarked_1',
       'Embarked_2', 'Embarked_3', 'Pclass_1', 'Pclass_2', 'Pclass_3',
       'Age Group_0-18', 'Age Group_19-30', 'Age Group_31-40',
       'Age Group_41-50', 'Age Group_51-60', 'Age Group_61-70',
       'Age Group_71-80'],
      dtype='object')

In [177]:
import pandas as pd
train_num = train_plne.drop(drop_columns_train, axis=1)
test_num = test_plne.drop(drop_columns_test, axis=1)
X = train_num.drop(['Survived','PassengerId'], axis=1)
y = train_num['Survived'].to_frame()

In [178]:
print(X.columns)
print(test_num.columns)

Index(['Sex_female', 'Sex_male', 'Embarked_1', 'Embarked_2', 'Embarked_3',
       'Pclass_1', 'Pclass_2', 'Pclass_3', 'Age Group_0-18', 'Age Group_19-30',
       'Age Group_31-40', 'Age Group_41-50', 'Age Group_51-60',
       'Age Group_61-70', 'Age Group_71-80'],
      dtype='object')
Index(['PassengerId', 'Sex_female', 'Sex_male', 'Embarked_1', 'Embarked_2',
       'Embarked_3', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Age Group_0-18',
       'Age Group_19-30', 'Age Group_31-40', 'Age Group_41-50',
       'Age Group_51-60', 'Age Group_61-70', 'Age Group_71-80'],
      dtype='object')


In [179]:
import torch
xs = torch.Tensor(X.values)
ys = torch.Tensor(y.values)
xs[0]

tensor([0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.])

In [221]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
# Create a TensorDataset and DataLoader
X_train, X_val, y_train, y_val = train_test_split(xs, ys, test_size=0.2, random_state=42)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False) #no need to shuffle validation.

In [222]:
len(xs[0])

15

In [223]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.relu = nn.ReLU()  # ReLU activation function
        self.fc2 = nn.Linear(hidden_size, output_size) # Second fully connected layer
        self.sigmoid = nn.Sigmoid() #Sigmoid activation for binary classification

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Instantiate the model
input_size = len(xs[0])
hidden_size = len(xs[0]) + 2
output_size = 1  # Binary classification, so one output neuron
model = SimpleNN(input_size, hidden_size, output_size)

In [224]:
import torch.optim as optim
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer

In [226]:
epochs = 300

for epoch in range(epochs):
    # Training
    model.train()  # Set the model to training mode
    for inputs, labels in train_dataloader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs).squeeze(0) #squeeze to remove extra dimension.

        # Calculate the loss
        loss = criterion(outputs, labels.squeeze(0)) #convert labels to float

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculations during validation
        for inputs, labels in val_dataloader:
            outputs = model(inputs).squeeze(0)
            val_loss += criterion(outputs, labels.squeeze(0)).item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.squeeze(0)).sum().item()

    avg_val_loss = val_loss / len(val_dataloader)
    accuracy = 100 * correct / total

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%')

print('Training finished!')

Epoch [10/300], Train Loss: 0.3635, Val Loss: 0.4834, Accuracy: 78.21%
Epoch [20/300], Train Loss: 0.5098, Val Loss: 0.4804, Accuracy: 79.89%
Epoch [30/300], Train Loss: 0.3833, Val Loss: 0.4834, Accuracy: 78.21%
Epoch [40/300], Train Loss: 0.5291, Val Loss: 0.4856, Accuracy: 78.77%
Epoch [50/300], Train Loss: 0.3440, Val Loss: 0.4884, Accuracy: 77.09%
Epoch [60/300], Train Loss: 0.4712, Val Loss: 0.4885, Accuracy: 79.89%
Epoch [70/300], Train Loss: 0.3627, Val Loss: 0.4921, Accuracy: 77.09%
Epoch [80/300], Train Loss: 0.2522, Val Loss: 0.4875, Accuracy: 78.77%
Epoch [90/300], Train Loss: 0.3236, Val Loss: 0.4883, Accuracy: 80.45%
Epoch [100/300], Train Loss: 0.3851, Val Loss: 0.4872, Accuracy: 79.33%
Epoch [110/300], Train Loss: 0.5274, Val Loss: 0.4949, Accuracy: 77.09%
Epoch [120/300], Train Loss: 0.2834, Val Loss: 0.4876, Accuracy: 79.89%
Epoch [130/300], Train Loss: 0.3185, Val Loss: 0.4838, Accuracy: 79.33%
Epoch [140/300], Train Loss: 0.4299, Val Loss: 0.4919, Accuracy: 77.65%
E

In [227]:
model.eval()  # Set to evaluation mode

SimpleNN(
  (fc1): Linear(in_features=15, out_features=17, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=17, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [228]:
input_features = test_num.drop('PassengerId', axis=1).iloc[5].values
len(input_features)

15

In [229]:
import torch

# Example input data
new_input = torch.Tensor(input_features) 

# Make predictions
with torch.no_grad():
    predictions = model(new_input)

# Process the predictions
predicted_label = (predictions > 0.5).float()  # Binary classification example
print("Predictions:", predictions)
print("Predicted label:", predicted_label)

Predictions: tensor([0.2721])
Predicted label: tensor([0.])


In [230]:
test_dataloader = TensorDataset(torch.Tensor(test_num.drop('PassengerId', axis=1).values))

In [231]:
ids = []
predictions = []
with torch.no_grad():  # Disable gradient calculations
    for (inputs,id) in zip(test_dataloader, test['PassengerId'].values):
        inputs = inputs[0] #dataloader returns a tuple, so we need to grab the first element.
        outputs = model(inputs).squeeze(0)  # Get model outputs
        predicted = (outputs > 0.5).float()  # Apply threshold for binary classification
        predictions.append(predicted.item())  # Collect predictions

# --- Processing predictions ---
print("Predictions:", predictions)

Predictions: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0

In [232]:
ids = []
predictions = []
for (X,id) in zip(test_dataloader, test['PassengerId'].values):
    label = model(X[0])
    ids.append(id)
    predictions.append((int((label > 0.5).float().sum().item())))

In [233]:
# Create a DataFrame from the lists
df = pd.DataFrame({
    'PassengerId': ids,
    'Survived': predictions
})

# Save the DataFrame to a CSV file without the default index
df.to_csv('submission.csv', index=False)